In [2]:
conda install pandas

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\kesha\anaconda3\envs\temporal_model_env

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.5           |   py38h080aedc_0         106 KB
    numexpr-2.8.4              |   py38h7b80656_1         128 KB
    pandas-2.0.3               |   py38h4ed8f06_0         9.8 MB
    python-tzdata-2023.3       |     pyhd3eb1b0_0         140 KB
    ------------------------------------------------------------
                                           Total:        10.2 MB

The following NEW packages will be INSTALLED:

  bottleneck         pkgs/main/win-64::bottleneck-1.3.5-py38h080aedc_0 
  numexpr            pkgs/main/win-64::numexpr-2.8.4-py38h7b80656_1 
  pandas             pkgs/main/win-64::pandas-2.0.3-py38h4ed8f0



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.2
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0




In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Define the Dataset class
class CSVDataset(Dataset):
    def __init__(self, filepath):
        self.data = pd.read_csv(filepath)
        self.X = torch.from_numpy(self.data.iloc[:, :-1].values).float()
        self.y = torch.from_numpy(self.data.iloc[:, -1].values).unsqueeze(1).float()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the neural network class
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Assume input features are 10 as per the synthetic data
        self.fc1 = torch.nn.Linear(10, 8)
        self.fc2 = torch.nn.Linear(8, 6)
        self.fc3 = torch.nn.Linear(6, 4)
        self.fc4 = torch.nn.Linear(4, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Load the dataset
dataset = CSVDataset(filepath='synthetic_data.csv')

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Instantiate the model
model = Net()

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):  # let's say we run for 5 epochs
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}: loss={running_loss/len(dataloader)}')

# Save the trained model
#torch.save(model.state_dict(), '/model.pth')

Epoch 1: loss=0.6927185263484716
Epoch 2: loss=0.6920402497053146
Epoch 3: loss=0.6921860687434673
Epoch 4: loss=0.6920996755361557
Epoch 5: loss=0.6923694536089897
